In [110]:
import json
import os
import h5py
import torch
from torchbiggraph.model import ComplexDiagonalDynamicOperator, DotComparator

with open("./data/statml/entity_names_user_id_0.json", "rt") as tf:
    dictionary = json.load(tf)

def perform_prediction(source_id, target_id):
    
    try:
        offset_source = dictionary.index(source_id)
        #print("our offset for source_id " , source_id, " is: ", offset_source)
        offset_target = dictionary.index(target_id)
        #print("our offset for target_id " , target_id, " is: ", offset_target)
        
        with h5py.File("./model/statml_ranking_50_epochs/embeddings_user_id_0.v52.h5", "r") as hf:
            embedding_source = hf["embeddings"][offset_source, :]
            embedding_target = hf["embeddings"][offset_target, :]


        comparator = DotComparator()
        comparator = DotComparator()
        score, _, _= comparator(
            comparator.prepare(torch.tensor(embedding_source.reshape([1,1,1024]))),
            comparator.prepare(torch.tensor(embedding_target.reshape([1,1,1024]))),
            torch.empty(1,0,1024),
            torch.empty(1,0,1024),
        )


        return score
    except:
        print("not in list")
        return 0
        





In [111]:
def get_test_samples():
    with open('./data/test-public.csv') as test:
        test_edges = []
        for line in test:
            edge_list = line.split()
            try:
                test_edges.append((int(edge_list[1]), int(edge_list[2])))
            except:
                pass
        return test_edges
        
test_samples = get_test_samples()

In [112]:
y_pred = []

In [113]:
n = len(test_samples)
i = 0
for i in range(0, n):
    sample = test_samples[i]
    pred = perform_prediction(str(sample[0]), str(sample[1]))
    pred = pred.item()
    y_pred.append(pred)
    #print(pred)
    
    i += 1

In [114]:
import numpy as np

In [115]:
def sigmoid(x):
    """ Sigmoid activation function
    :param x: scalar value
    :return: sigmoid activation
    """
    return 1 / (1 + np.exp(-x))

In [116]:
proba = []

In [117]:
for element in y_pred:
    prob = sigmoid(element)
    proba.append(prob)

In [118]:
import csv
with open("jupiter_logistic_biggraph.csv","w",newline="") as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(["Id","Predictions"])
    test_id=1
    for prediction in proba:
        writer.writerow([test_id,prediction])
        test_id+=1